In [9]:
import pandas as pd

In [10]:
df=pd.read_csv('FakeNewsNet.csv')

In [11]:
df.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [12]:
df.shape

(23196, 5)

In [13]:
df.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [14]:
df.isnull().sum()

,0
title,0
news_url,330
source_domain,330
tweet_num,0
real,0


In [15]:
###Drop Nan Values
df=df.dropna()


In [22]:
import tensorflow as tf

In [23]:
tf.__version__

'2.19.0'

In [76]:
display(df['real'].value_counts())

,count
real,
1,17371
0,5495


In [77]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X, y)

In [78]:
import pandas as pd
display(pd.Series(y_resampled).value_counts())

,count
real,
1,17371
0,17371


In [83]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
import numpy as np # Import numpy here

# Vocabulary size
voc_size = 5000

# One-hot encode the corpus
onehot_repr = [one_hot(words, voc_size) for words in corpus]

# Set sentence length
sent_length = 20

# Pad sequences
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)

# Create the LSTM model
embedding_vector_features = 40  # Adjust as needed
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features))
model.add(Dropout(0.3))
model.add(LSTM(100)) # Reduced LSTM units
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid')) # Output layer for binary classification
model.build(input_shape=(None,sent_length))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 20, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 20, 40)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 100)            │        56,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 256,501 (1001.96 KB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

In [84]:
### Dataset Preprocessing
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer ##stemming purpose

nltk.download('stopwords')

ps = PorterStemmer()
corpus = []

# Use the resampled data for preprocessing
messages_resampled = X_resampled['title']
messages_resampled.reset_index(inplace=True, drop=True) # Reset and drop old index

for i in range(0, len(messages_resampled)):
    review = re.sub('[^a-zA-Z]', ' ', messages_resampled[i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [85]:
import numpy as np
from sklearn.model_selection import train_test_split

# Use the resampled data (embedded_docs, y_resampled) for training
X_final = np.array(embedded_docs)
y_final = np.array(y_resampled) # Use y_resampled

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64) # Reduced epochs and increased batch size

Epoch 1/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.6675 - loss: 0.5853 - val_accuracy: 0.7991 - val_loss: 0.4326
Epoch 2/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.8311 - loss: 0.3867 - val_accuracy: 0.8151 - val_loss: 0.4103
Epoch 3/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8565 - loss: 0.3413 - val_accuracy: 0.8292 - val_loss: 0.3970
Epoch 4/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8719 - loss: 0.3099 - val_accuracy: 0.8432 - val_loss: 0.3830
Epoch 5/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.8957 - loss: 0.2627 - val_accuracy: 0.8502 - val_loss: 0.3879
Epoch 6/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9086 - loss: 0.2293 - val_accuracy: 0.8587 - val_loss: 0.3879
Epoch 7/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9237 - loss: 0.2003 - val_accuracy: 0.8612 - val_loss: 0.4004
Epoch 8/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9299 - loss: 0.1844 - val_accuracy

In [86]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Predict on the test data
y_pred = model.predict(X_test)
y_pred_classes = np.where(y_pred > 0.5, 1, 0) # Using 0.5 as the threshold

# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred_classes)
display(cm)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_classes)
print(f"Accuracy: {accuracy}")

# Generate classification report
report = classification_report(y_test, y_pred_classes)
print(report)

359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


array([[5283,  431],
       [1055, 4696]])

Accuracy: 0.8703881378107283
              precision    recall  f1-score   support

           0       0.83      0.92      0.88      5714
           1       0.92      0.82      0.86      5751

    accuracy                           0.87     11465
   macro avg       0.87      0.87      0.87     11465
weighted avg       0.87      0.87      0.87     11465

